# Training with Opensoundscape & Pytorch Lightning

OpenSoundscape provides classes that support the use of Pytorch Lightning's Trainer class, which implements various training techniques, speedups, and utilities. To use Lightning, simply use the `opensoundscape.ml.lightning.LightningSpectrogramModule` class rather than the `opensoundscape.ml.cnn.SpectrogramClassifier` class (or `CNN` class, which is now an alias for `SpectrogramClassifier`). For the most part, the API and functionality is similar to the pure-pytorch classes, with a few major differences:
- to train, call the `.fit_with_trainer()` method ("train()" method is reserved for other purposes when using Lightning). Pass any kwargs to lightning.Trainer()to customize the Lightning Trainer. 
- to predict, call `.predict_with_trainer()`, passing any kwargs for the lightning.Trainer init with `lightning_trainer_kwargs=dict(...)`
- note that with the Lightning Trainer, you can use various logging platforms, while only Weights and Biases is currently supported in the pure PyTorch classes

Check out the lightning.Trainer [docs](https://lightning.ai/docs/pytorch/stable/common/trainer.html) for the full set of implemented features.

In [1]:
# if this is a Google Colab notebook, install opensoundscape in the runtime environment
if 'google.colab' in str(get_ipython()):
  %pip install git+https://github.com/kitzeslab/opensoundscape@develop ipykernel==5.5.6 ipython==7.34.0 pillow==9.4.0
  num_workers=0
else:
  num_workers=4

## Setup

### Import needed packages

In [2]:
# the cnn module provides classes for training/predicting with various types of CNNs
from opensoundscape import CNN

#other utilities and packages
import torch
import pandas as pd
from pathlib import Path
import numpy as np
import pandas as pd
import random 
import subprocess
from glob import glob
import sklearn

#set up plotting
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize']=[15,5] #for large visuals
%config InlineBackend.figure_format = 'retina'

### Set random seeds

Set manual seeds for Pytorch and Python. These essentially "fix" the results of any stochastic steps in model training, ensuring that training results are reproducible. You probably don't want to do this when you actually train your model, but it's useful for debugging.

In [3]:
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

### Download files

Training a machine learning model requires some pre-labeled data. These data, in the form of audio recordings or spectrograms, are labeled with whether or not they contain the sound of the species of interest. 

These data can be obtained from online databases such as Xeno-Canto.org, or by labeling one's own ARU data using a program like Cornell's Raven sound analysis software. In this example we are using a set of annotated avian soundscape recordings that were annotated using the software Raven Pro 1.6.4 (Bioacoustics Research Program 2022):

<blockquote><i>An annotated set of audio recordings of Eastern North American birds containing frequency, time, and species information. </i> Lauren M. Chronister,  Tessa A. Rhinehart,  Aidan Place,  Justin Kitzes.
https://doi.org/10.1002/ecy.3329 
</blockquote>

These are the same data that are used by the annotation and preprocessing tutorials, so you can skip this step if you've already downloaded them there.

### Download example files
Download a set of example audio files and Raven annotations:

Option 1: run the cell below

- if you get a 403 error, DataDryad suspects you are a bot. Use Option 2. 

Option 2:

- Download and unzip both `annotation_Files.zip` and `mp3_Files.zip` from the https://datadryad.org/stash/dataset/doi:10.5061/dryad.d2547d81z  
- Move the unzipped contents into a subfolder of the current folder called `./annotated_data/`

In [ ]:
# Note: the "!" preceding each line below allows us to run bash commands in a Jupyter notebook
# If you are not running this code in a notebook, input these commands into your terminal instead
!wget -O annotation_Files.zip https://datadryad.org/stash/downloads/file_stream/641805;
!wget -O mp3_Files.zip https://datadryad.org/stash/downloads/file_stream/641807;
!mkdir annotated_data;
!unzip annotation_Files.zip -d ./annotated_data/annotation_Files;
!unzip mp3_Files.zip -d ./annotated_data/mp3_Files;

--2024-08-28 12:30:10--  https://datadryad.org/stash/downloads/file_stream/641805
Resolving datadryad.org (datadryad.org)... 34.218.192.228, 52.27.226.201, 34.223.182.183, ...
Connecting to datadryad.org (datadryad.org)|34.218.192.228|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-08-28 12:30:10 ERROR 403: Forbidden.

--2024-08-28 12:30:11--  https://datadryad.org/stash/downloads/file_stream/641807
Resolving datadryad.org (datadryad.org)... 52.27.226.201, 34.223.182.183, 35.80.241.54, ...
Connecting to datadryad.org (datadryad.org)|52.27.226.201|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-08-28 12:30:11 ERROR 403: Forbidden.

mkdir: annotated_data: File exists
Archive:  annotation_Files.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(

## Prepare training and validation data

To prepare audio data for machine learning, we need to convert our annotated data into clip-level labels.

These steps are covered in depth in other tutorials, so we'll just set our clip labels up quickly for this example.

First, get exactly matched lists of audio files and their corresponding selection files:

In [31]:
# Set the current directory to where the dataset is downloaded
dataset_path = Path("./annotated_data/")

# Make a list of all of the selection table files
selection_files = glob(f"{dataset_path}/annotation_Files/*/*.txt")

# Create a list of audio files, one corresponding to each Raven file
# (Audio files have the same names as selection files with a different extension)
audio_files = [f.replace('annotation_Files','mp3_Files').replace('.Table.1.selections.txt','.mp3') for f in selection_files]

#Next, convert the selection files and audio files to a `BoxedAnnotations` object, which contains
#the time, frequency, and label information for all annotations for every recording in the dataset.

from opensoundscape.annotations import BoxedAnnotations
# Create a dataframe of annotations
annotations = BoxedAnnotations.from_raven_files(
    selection_files,
    audio_files)

# Parameters to use for label creation
clip_duration = 3
clip_overlap = 0
min_label_overlap = 0.25
species_of_interest = ["NOCA", "EATO", "SCTA", "BAWW", "BCCH", "AMCR", "NOFL"]

# Create dataframe of one-hot labels
clip_labels = annotations.clip_labels(
    clip_duration = clip_duration, 
    clip_overlap = clip_overlap,
    min_label_overlap = min_label_overlap,
    class_subset = species_of_interest # You can comment this line out if you want to include all species.
)

from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(clip_labels, test_size=0.2)

/Users/SML161/opensoundscape/opensoundscape/annotations.py:216: UserWarning: annotated_data/annotation_Files/Recording_1/Recording_1_Segment_01.Table.1.selections.txt has zero rows.
  warnings.warn(f"{raven_file} has zero rows.")


### Create Lightning-copmatible model

Now, create a LightningSpectrogramModule object, which integrates OpenSoundscape with Pytorch Lightning's powerful Trainer class

In [32]:
# Create a CNN object designed to recognize 3-second samples
from opensoundscape.ml.lightning import LightningSpectrogramModule

# initializing it looks the same as for the CNN class.
# Let's use resnet34 architecture and 3s clip duration
model = LightningSpectrogramModule(
    architecture = 'resnet34',
    classes = clip_labels.columns.tolist(),
    sample_duration = 3 
)

## Train with Lightning

Lightning will take a bit of time to get things set up. After that, it can be substantially faster than training in pure PyTorch.

In [33]:
# again, the API is very similar to CNN
# but now, we can pass any kwargs to Lightning.Trainer() as well. For example, 
# let's use the `accum_grad_batches` argument to accumulate gradients over 2 batches before running the optimizer,
# effectively doubling the batch size.
model.fit_with_trainer(train_df, val_df, epochs=4, batch_size=32, num_workers=num_workers, accumulate_grad_batches=2)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/SML161/miniconda3/envs/opso_dev/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
Missing logger folder: ./lightning_logs
/Users/SML161/miniconda3/envs/opso_dev/lib/python3.9/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:653: Checkpoint directory /Users/SML161/opensoundscape/docs/tutorials exists and is not empty.
/Users/SML161/miniconda3/envs/opso_dev/lib/pytho

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/SML161/miniconda3/envs/opso_dev/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
/Users/SML161/miniconda3/envs/opso_dev/lib/python3.9/site-packages/torchmetrics/functional/classification/precision_recall_curve.py:798: UserWarning: MPS: nonzero op is supported natively starting from macOS 13.0. Falling back on CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Indexing.mm:334.)
  unique_mapping = unique_mapping[unique_mapping >= 0]
/Users/SML161/miniconda3/envs/opso_dev/lib/python3.9/site-packages/torchmetrics/functional/classification/average_precision.py:308: UserWarning: MPS: no support for int64 for sum_out_mps, downcasting to a smaller data type (int32/float32). Native support for int64 has been added in macOS 13.3. (Triggered in

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=4` reached.


Training complete
Best model with score 0.140 is saved to /Users/SML161/opensoundscape/docs/tutorials/epoch=1-step=194.ckpt
0 of 6160 total training samples failed to preprocess


## run inference

In [34]:
model.predict_with_trainer(val_df, batch_size=32, num_workers=num_workers)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/SML161/miniconda3/envs/opso_dev/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'predict_dataloader' to speed up the dataloader worker initialization.


Predicting: |          | 0/? [00:00<?, ?it/s]

,,,NOCA,EATO,SCTA,BAWW,BCCH,AMCR,NOFL
file,start_time,end_time,,,,,,,
annotated_data/mp3_Files/Recording_1/Recording_1_Segment_13.mp3,84.0,87.0,11.701532,10.014467,9.671937,1.633159,9.227418,8.859664,-0.052373
annotated_data/mp3_Files/Recording_4/Recording_4_Segment_13.mp3,87.0,90.0,9.945120,8.878679,9.042792,1.747799,8.751578,8.416622,0.018097
annotated_data/mp3_Files/Recording_1/Recording_1_Segment_07.mp3,249.0,252.0,9.504481,8.450200,8.734877,1.783901,8.292206,8.354047,0.422760
annotated_data/mp3_Files/Recording_4/Recording_4_Segment_11.mp3,231.0,234.0,9.832380,9.031016,9.131749,1.859293,9.226156,8.416931,0.288400
annotated_data/mp3_Files/Recording_1/Recording_1_Segment_35.mp3,228.0,231.0,10.225933,8.867890,9.046994,1.748763,8.689975,8.404178,0.139670
...,...,...,...,...,...,...,...,...,...
annotated_data/mp3_Files/Recording_1/Recording_1_Segment_25.mp3,186.0,189.0,10.359131,9.550800,9.299773,2.005651,9.608356,8.805997,0.035945
annotated_data/mp3_Files/Recording_1/Recording_1_Segment_27.mp3,138.0,141.0,10.066076,9.293936,9.277360,1.741633,8.826027,8.666476,-0.180602
annotated_data/mp3_Files/Recording_2/Recording_2_Segment_14.mp3,90.0,93.0,9.545153,8.941689,8.957946,1.810180,8.933350,8.355339,0.146811


## Next steps: 
experiment with the various optimizations and features of lightning.Trainer, such as integration with several different logging platforms, multi-device distributed training, and more. 

Check out the Lightning Trainer [docs](https://lightning.ai/docs/pytorch/stable/common/trainer.html) to learn more. 

## clean up

In [39]:
import shutil

# uncomment to remove the training files
# shutil.rmtree('./annotated_data')

shutil.rmtree('./wandb', ignore_errors=True)
shutil.rmtree('./model_training_checkpoints', ignore_errors=True)
for f in glob('./*.ckpt'):
    Path(f).unlink()

try:
    Path('annotation_Files.zip').unlink()
except:
    pass
try:
    Path('mp3_Files.zip').unlink()
except:
    pass